# FP树的类定义

In [1]:
import copy

In [2]:
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode
        self.children = {}
        
    def inc(self, numOccur):
        self.count += numOccur
    
    def disp(self, ind=1):
        print('  '*ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind+1)

In [3]:
rootNode = treeNode('pyramid', 9, None)

In [4]:
rootNode.children['eye'] = treeNode('eye', 13, None)

In [5]:
rootNode.disp()

   pyramid   9
     eye   13


In [6]:
rootNode.children['phoenix'] = treeNode('phoenix', 3, None)
rootNode.disp()

   pyramid   9
     eye   13
     phoenix   3


# 构建FP树

In [7]:
def createTree(dataSet, minSup=1):
    headerTable = {}
    for trans in dataSet:  #第一遍遍历扫描数据集并统计每个元素项出现的频度
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    ##########################之前这里出错了，加上这一语句就好了
    headerTab = copy.deepcopy(headerTable)
    ###########################copy 模块的 deepcopy 方法，完全拷贝了父对象及其子对象。
    for k in headerTable.keys():
        if headerTable[k] < minSup:  #删掉出现次数少于minSup的元素项
            del(headerTable[k])
    freqItemSet = set(headerTable.keys())
    if len(freqItemSet) == 0:
        return None, None  #如果所有项都不频繁，则退出
    for k in headerTable:
        headerTable[k] = [headerTable[k], None]
    retTree = treeNode('Null Set', 1, None)  #创建只包含空集合的根节点
    for tranSet, count in dataSet.items():  #第二遍遍历
        localD = {}
        for item in tranSet:
            if item in freqItemSet:  #只考虑频繁元素项
                localD[item] = headerTable[item][0]
        if len(lacalD) > 0:
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)]
            updateTree(orderedItem, retTree, headerTable, count)
    return retTree, headerTable

def updateTree(items, inTree, headerTable, count):
    if items[0] in inTree.children:  #如果作为子节点存在
        inTree.children[items[0]].inc(count)  #增加计数
    else:  #否则，创建一个新的treeNode并将其作为一个子节点添加到树中
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        if headerTable[items[0]][1] == None:
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    if len(items) > 1:
        updateTree(items[1::], inTree.children[items[0]], headerTable, count)  #对剩下的元素项迭代调用
        
def updateHeader(nodeToTest, targetNode):  #更新头指针表以指向新的节点
    while (nodeToTest.nodeLink != None):
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

## 复制的别人的代码，来查找bug

In [8]:
#FP树构建函数
#使用数据集和最小支持度作为参数构建FP树
#构建时，会遍历数据集两次，
#第一次统计每个元素项出现的频度，并将信息存储到头指针表中，并删除那些次数少于minSup的项
def createTree(dataSet, minSup=1):
    
    headerTable = {}         #将每个元素出现的频度的信息存储到headerTable
    #遍历两次数据集
    #第一次统计每个元素项出现的频度，并存储，然后删除那些次数少于minSup的项
    for trans in dataSet:
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    
    #删除那些次数少于minSup的项
    headerTab = copy.deepcopy(headerTable)
    for k in headerTab.keys():  
        if headerTab[k] < minSup: 
            del(headerTable[k])
    freqItemSet = set(headerTable.keys())
    #print('freqItemSet: ',freqItemSet)
    #若所有项都不频繁，则直接退出
    if len(freqItemSet) == 0: return None, None  
    for k in headerTable:
        headerTable[k] = [headerTable[k], None] #reformat headerTable to use Node link 
    #print('headerTable: ',headerTable)
    #创建FP树
    retTree = treeNode('Null Set', 1, None) 
    #第二次遍历数据集
    for tranSet, count in dataSet.items():  
        localD = {}
        #根据全局频率对每个事务中的元素进行排序
        for item in tranSet:  
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
        if len(localD) > 0:
            #按照元素项的绝对出现的次数大小进行排序,key=lambda p: p[1]代表按照字典的第2个元素进行排序
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)]
            #print("orderedItems:",orderedItems)
            #使用排序后的频率项集对树进行填充
            updateTree(orderedItems, retTree, headerTable, count)
    return retTree, headerTable #return tree and header table
 
    #输入的是一个项集，让FP树生长
def updateTree(items, inTree, headerTable, count):
    #测试事务中的第一个元素项是否作为子节点存在
    if items[0] in inTree.children:
        #若是，则增加该元素项的计数
        inTree.children[items[0]].inc(count) 
    else:   #否则，则创建一个新的treeNode，并将其作为一个子节点添加到树中
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        #更新头指针表
        if headerTable[items[0]][1] == None:  
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    #不断调用调用自身，每次调用时会去掉列表中的第一个元素，直到调用完所有元素
    if len(items) > 1:
        updateTree(items[1::], inTree.children[items[0]], headerTable, count)
 
    #更新头指针表，确保节点链接指向树中该元素项的每一个实例        
def updateHeader(nodeToTest, targetNode):   
    while (nodeToTest.nodeLink != None):    
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode


# 数据集转换

In [9]:
def loadSimpDat():
    simpDat = [['r', 'z', 'h', 'j', 'p'],
              ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
              ['z'],
              ['r', 'x', 'n', 'o', 's'],
              ['y', 'r', 'x', 'z', 'q', 't', 'p'],
              ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpDat

def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict

In [10]:
simpDat = loadSimpDat()
simpDat

[['r', 'z', 'h', 'j', 'p'],
 ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
 ['z'],
 ['r', 'x', 'n', 'o', 's'],
 ['y', 'r', 'x', 'z', 'q', 't', 'p'],
 ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]

In [11]:
initSet = createInitSet(simpDat)
initSet

{frozenset({'h', 'j', 'p', 'r', 'z'}): 1,
 frozenset({'s', 't', 'u', 'v', 'w', 'x', 'y', 'z'}): 1,
 frozenset({'z'}): 1,
 frozenset({'n', 'o', 'r', 's', 'x'}): 1,
 frozenset({'p', 'q', 'r', 't', 'x', 'y', 'z'}): 1,
 frozenset({'e', 'm', 'q', 's', 't', 'x', 'y', 'z'}): 1}

In [12]:
myFPtree, myHeaderTab = createTree(initSet, 3)

In [13]:
myFPtree.disp()

   Null Set   1
     z   5
       r   1
       x   3
         t   3
           s   2
             y   2
           r   1
             y   1
     x   1
       r   1
         s   1


# 从FP树中挖掘频繁项集

## 原理和Apriori算法大致类似，从单元素项集开始

## 发现以给定元素项结尾的所有路径的函数

In [18]:
def ascendTree(leafNode, prefixPath):
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)
        
def findPrefixPath(basePat, treeNode):
    condPats = {}
    while treeNode != None:
        prefixPath = []
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1:
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats

In [19]:
findPrefixPath('x', myHeaderTab['x'][1])

{frozenset({'z'}): 3}

In [20]:
findPrefixPath('z', myHeaderTab['z'][1])

{}

In [21]:
findPrefixPath('r', myHeaderTab['r'][1])

{frozenset({'z'}): 1, frozenset({'x'}): 1, frozenset({'t', 'x', 'z'}): 1}

## 根据条件模式基来创建条件树

In [22]:
def mineTree(inTree,headerTable, minSup,preFix, freqItemList):
    #对头指针表中的元素项按照其出现频率从小到大进行排序
    bigL = [v[0] for v in sorted(headerTable.items(),key = lambda p:p[1][0])]
    for basePat in bigL:
        newFreqSet = preFix.copy()
        newFreqSet.add(basePat)
        #将每一个频繁项添加到频繁项集列表freqItemList中
        freqItemList.append(newFreqSet)
        #创建条件基
        condPattBases = findPrefixPath(basePat, headerTable[basePat][1])
        #利用条件基当做一个新数据集输送给createTree()函数
        myCondTree,myHead = createTree(condPattBases,minSup)
        #只要树中有元素项，递归调用mineTree()函数
        if myHead != None:
            print("condition tree for:",newFreqSet)
            myCondTree.disp(1)
            mineTree(myCondTree,myHead,minSup,newFreqSet,freqItemList)

In [23]:
freqItems = []
mineTree(myFPtree, myHeaderTab, 3, set([]), freqItems)

condition tree for: {'t'}
   Null Set   1
     z   3
       x   3
condition tree for: {'t', 'x'}
   Null Set   1
     z   3
condition tree for: {'s'}
   Null Set   1
     x   3
condition tree for: {'y'}
   Null Set   1
     t   3
       z   3
         x   3
condition tree for: {'y', 'z'}
   Null Set   1
     t   3
condition tree for: {'y', 'x'}
   Null Set   1
     t   3
       z   3
condition tree for: {'y', 'z', 'x'}
   Null Set   1
     t   3
condition tree for: {'x'}
   Null Set   1
     z   3


In [24]:
freqItems

[{'r'},
 {'t'},
 {'t', 'z'},
 {'t', 'x'},
 {'t', 'x', 'z'},
 {'s'},
 {'s', 'x'},
 {'y'},
 {'t', 'y'},
 {'y', 'z'},
 {'t', 'y', 'z'},
 {'x', 'y'},
 {'t', 'x', 'y'},
 {'x', 'y', 'z'},
 {'t', 'x', 'y', 'z'},
 {'x'},
 {'x', 'z'},
 {'z'}]

# 新闻网站点击流中挖掘

In [26]:
parsedDat = [line.split() for line in open('kosarak.dat').readlines()]
initSet = createInitSet(parsedDat)
myFPtree, myHeaderTab = createTree(initSet, 100000)

In [27]:
myFreqList = []
mineTree(myFPtree, myHeaderTab, 100000, set([]), myFreqList)

condition tree for: {'1'}
   Null Set   1
     6   107404
condition tree for: {'3'}
   Null Set   1
     6   186289
       11   117401
     11   9718
condition tree for: {'3', '11'}
   Null Set   1
     6   117401
condition tree for: {'11'}
   Null Set   1
     6   261773


In [28]:
myFreqList

[{'1'},
 {'1', '6'},
 {'3'},
 {'11', '3'},
 {'11', '3', '6'},
 {'3', '6'},
 {'11'},
 {'11', '6'},
 {'6'}]